In [59]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /Users/Dhyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
import pandas as pd
import numpy as np
import re
import warnings
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [46]:
f= open("Review.txt","w+")

In [47]:
for i in range(1):
     f.write("I took my father out for dinner to Le Bistro on New Year’s Eve. The décor and service were fantastic. We enjoyed the food, especially their French countryside specials and their Chardonnay collections. However, my father thought the menu prices were a bit on the high side. Valet parking was also expensive. Overall, we definitely recommend Le Bistro for special occasions!”Overall rating: 8 stars out of 10 %d\r\n" % (i+0))

In [48]:
def openFile(path):
    #param path: path/to/file.ext (str)
    #Returns contents of file (str)
    with open(path) as file:
        data = file.read()
    return data
_data = openFile('/Users/Dhyan/NLP/FinalExam/Review.txt')
#txt = openFile('/Users/Dhyan/Applications/NLP_Final/FinalExam.txt').read_text(decoding="utf-8")

In [49]:
datasets = [_data]

combined_dataset = []
# separate samples from each other
for dataset in datasets:
    combined_dataset.extend(dataset.split('\n'))

# separate each label from each sample
dataset = [sample.split('\t') for sample in combined_dataset]

In [50]:
df = pd.DataFrame(data=dataset, columns=['Reviews'])

In [51]:
# Remove any blank reviews
df = df[df["Reviews"].notnull()]

# shuffle the dataset for later.
# Note this isn't necessary (the dataset is shuffled again before used), 
# but is good practice.
df = df.sample(frac=1)

In [52]:
import string

df['Word Count'] = [len(review.split()) for review in df['Reviews']]

df['Uppercase Char Count'] = [sum(char.isupper() for char in review) \
                              for review in df['Reviews']]                           

df['Special Char Count'] = [sum(char in string.punctuation for char in review) \
                            for review in df['Reviews']]  

In [54]:
Positive = df['Word Count'].describe()
Uppercase = df['Uppercase Char Count'].describe()
Lower = df['Special Char Count'].describe()

In [55]:
Positive

count     2.000000
mean     34.000000
std      48.083261
min       0.000000
25%      17.000000
50%      34.000000
75%      51.000000
max      68.000000
Name: Word Count, dtype: float64

In [56]:
Uppercase

count     2.000000
mean      8.000000
std      11.313708
min       0.000000
25%       4.000000
50%       8.000000
75%      12.000000
max      16.000000
Name: Uppercase Char Count, dtype: float64

In [57]:
Lower

count     2.000000
mean      5.000000
std       7.071068
min       0.000000
25%       2.500000
50%       5.000000
75%       7.500000
max      10.000000
Name: Special Char Count, dtype: float64

In [58]:
from collections import Counter

def getMostCommonWords(reviews, n_most_common, stopwords=None):
    # param reviews: column from pandas.DataFrame (e.g. df['Reviews']) 
        #(pandas.Series)
    # param n_most_common: the top n most common words in reviews (int)
    # param stopwords: list of stopwords (str) to remove from reviews (list)
    # Returns list of n_most_common words organized in tuples as 
        #('term', frequency) (list)

    # flatten review column into a list of words, and set each to lowercase
    flattened_reviews = [word for review in reviews for word in \
                         review.lower().split()]


    # remove punctuation from reviews
    flattened_reviews = [''.join(char for char in review if \
                                 char not in string.punctuation) for \
                         review in flattened_reviews]


    # remove stopwords, if applicable
    if stopwords:
        flattened_reviews = [word for word in flattened_reviews if \
                             word not in stopwords]


    # remove any empty strings that were created by this process
    flattened_reviews = [review for review in flattened_reviews if review]

    return Counter(flattened_reviews).most_common(n_most_common)

In [63]:
getMostCommonWords(df['Reviews'], 10)

[('the', 4),
 ('my', 2),
 ('father', 2),
 ('out', 2),
 ('for', 2),
 ('le', 2),
 ('bistro', 2),
 ('on', 2),
 ('and', 2),
 ('were', 2)]

In [64]:
getMostCommonWords(df['Reviews'], 10, stopwords.words('english'))

[('father', 2),
 ('le', 2),
 ('bistro', 2),
 ('took', 1),
 ('dinner', 1),
 ('new', 1),
 ('year’s', 1),
 ('eve', 1),
 ('décor', 1),
 ('service', 1)]

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
bow = vectorizer.fit_transform(df['Reviews'])


In [ ]:
labels = df['Labels']